In [1]:

import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from itertools import chain
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from src.train import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

backtest_frame_path = '/home/liuqize/QuantFrame'
sys.path.append(backtest_frame_path)
try:
    from MLMethods import MLData, MLTrain, LGBM_EvalMetrics
    from Factors import TSFeatureEngineer
except:
    error_msg = 'Please check the path of backtest_frame'
    print(error_msg)

data_path = '/data/user_home/liuqize/aft_data'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
key_col = ['date','time','sym']
target_cols =  ['label_5', 'label_10', 'label_20', 'label_40','label_60']
data_path = '/data/user_home/liuqize/aft_data'

# 读取数据

## raw feature and label data

In [71]:

feature_1 = pd.read_parquet(os.path.join(data_path, 'original.parquet'))
feature_2 = pd.read_parquet(os.path.join(data_path, 'important_features.parquet'))

In [ ]:
# target_df
from glob import glob
target_df = pd.concat([pd.read_csv(f) for f in glob(os.path.join(data_path,'raw_data/train/*'))])

target_df.reset_index(drop=True, inplace=True)
target_df = target_df[['date','time','sym'] + target_cols]

In [ ]:
df = pd.concat([
    feature_1.set_index(key_col),
    feature_2.set_index(key_col),
    target_df.set_index(key_col)
], axis=1).reset_index()

In [ ]:
# 保存df, 方便用pipeline
df.to_pickle(os.path.join(data_path, 'all_data.pkl'))

## preprocessed data

In [2]:
df = pd.read_pickle(os.path.join(data_path, 'all_data.pkl'))

In [3]:
df.to_parquet(os.path.join(data_path, 'all_data.parquet'))

In [4]:
features_cols = df.columns[3:-5].tolist()

In [ ]:
len(features_cols)

427

In [5]:
target = target_cols[0]
feature2num = {col: i for i, col in enumerate(features_cols)}
num2feature = {i: col for i, col in enumerate(features_cols)}

new_feature_cols = list(range(len(features_cols)))
df.rename(columns=feature2num, inplace=True)
df['time_pk'] = df['date'].astype(str).str.cat(df['time'],sep='_')
train_df = df.query('date < 50')
test_df = df.query('date >= 50')


In [57]:
# 取少量数据出来做cv
SAMPLE_FRACTION = 0.1
lab_df_list = []
for lab in train_df[target].unique():
    lab_df_list.append(train_df.query(f'{target} == {lab}'))

expected_smaple_num = int(len(train_df) * SAMPLE_FRACTION)

cv_df = pd.concat([
    df.sample(expected_smaple_num // 3, random_state=42) for df in lab_df_list
])
cv_df.reset_index(drop=True, inplace=True)

# cv_df = train_df.groupby(['date','sym']).sample(frac=0.1).copy()
# cv_df.reset_index(drop=True, inplace=True)
# cv_df.head(3)

In [58]:

# def train_lgbm(param_dict, train_dataset
index_list = MLData.ts_KFold_equal_time(cv_df,time_col='time_pk',asset_pk_col='sym',nsplit=5)

In [59]:
target='label_5'
study = MLTrain.create_optuna_study('tuning_'+target, save_path='./optuna_studies/',direction='maximize')

[I 2023-11-06 13:07:40,689] A new study created in RDB with name: tuning_label_5


In [60]:
params = {
    
    'objective': 'multiclass',  # 多分类任务
    'num_class': 3,  # 类别数，这里是三分类任务
    'metric': 'multi_logloss',  # 多分类损失函数
    'n_jobs': cpu_count()//2,
    'force_col_wise': True,
    'verbose': -1,
}


In [61]:

MLTrain.LGBMSearchCV_manual(
    cv_df,  
    x_cols=new_feature_cols,
    y_col=target,
    params=params,
    cv=index_list,
    feval = [LGBM_EvalMetrics.macro_f1],
    feval_name = 'macro_f1',
    study=study,
    n_trial = 10000
    
)

{'n_estimators': 10000, 'learning_rate': 0.2, 'num_leaves': 232, 'max_depth': 2, 'min_data_in_leaf': 50, 'lambda_l1': 0.282109923136379, 'lambda_l2': 0.7749250733183953, 'bagging_fraction': 0.9594643242075952, 'bagging_freq': 2, 'feature_fraction': 0.8538410985943087, 'max_bin': 57, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}


[I 2023-11-06 13:09:03,300] Trial 0 finished with value: 0.6489708904445122 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2, 'num_leaves': 232, 'max_depth': 2, 'min_data_in_leaf': 50, 'lambda_l1': 0.282109923136379, 'lambda_l2': 0.7749250733183953, 'bagging_fraction': 0.9594643242075952, 'bagging_freq': 2, 'feature_fraction': 0.8538410985943087, 'max_bin': 57, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}. Best is trial 0 with value: 0.6489708904445122.


{'n_estimators': 10000, 'learning_rate': 0.2, 'num_leaves': 137, 'max_depth': 3, 'min_data_in_leaf': 20, 'lambda_l1': 0.255740324509279, 'lambda_l2': 0.014790346093813002, 'bagging_fraction': 0.8216579290587394, 'bagging_freq': 6, 'feature_fraction': 0.9205065217336345, 'max_bin': 122, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}


[I 2023-11-06 13:09:33,266] Trial 1 finished with value: 0.6513760217902733 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2, 'num_leaves': 137, 'max_depth': 3, 'min_data_in_leaf': 20, 'lambda_l1': 0.255740324509279, 'lambda_l2': 0.014790346093813002, 'bagging_fraction': 0.8216579290587394, 'bagging_freq': 6, 'feature_fraction': 0.9205065217336345, 'max_bin': 122, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}. Best is trial 1 with value: 0.6513760217902733.


{'n_estimators': 10000, 'learning_rate': 0.2, 'num_leaves': 51, 'max_depth': 5, 'min_data_in_leaf': 200, 'lambda_l1': 0.33149243813806095, 'lambda_l2': 0.025162825849935355, 'bagging_fraction': 0.439073455883215, 'bagging_freq': 4, 'feature_fraction': 0.8632576195297157, 'max_bin': 79, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}


...

{'n_estimators': 10000, 'learning_rate': 0.2, 'num_leaves': 231, 'max_depth': 8, 'min_data_in_leaf': 50, 'lambda_l1': 0.025561973827297295, 'lambda_l2': 0.12571456895514965, 'bagging_fraction': 0.9312461874410187, 'bagging_freq': 5, 'feature_fraction': 0.7983645092932664, 'max_bin': 132, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}


[I 2023-11-06 18:15:15,385] Trial 3805 pruned. Trial was pruned at iteration 19.


{'n_estimators': 10000, 'learning_rate': 0.1, 'num_leaves': 145, 'max_depth': 8, 'min_data_in_leaf': 50, 'lambda_l1': 0.036570510183285936, 'lambda_l2': 0.9585792943110026, 'bagging_fraction': 0.6184097732064928, 'bagging_freq': 2, 'feature_fraction': 0.7617217810499433, 'max_bin': 219, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}


[I 2023-11-06 18:15:20,886] Trial 3806 pruned. Trial was pruned at iteration 0.


{'n_estimators': 10000, 'learning_rate': 0.1, 'num_leaves': 171, 'max_depth': 8, 'min_data_in_leaf': 200, 'lambda_l1': 0.008616964749509638, 'lambda_l2': 0.10514381021217498, 'bagging_fraction': 0.7873554649022345, 'bagging_freq': 4, 'feature_fraction': 0.7082642931675273, 'max_bin': 212, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}


## 测试代码

In [53]:
test_params = {'n_estimators': 10000, 'learning_rate': 0.01, 'num_leaves': 41, 'max_depth': 7, 'min_data_in_leaf': 1000, 'lambda_l1': 0.003724426825310446, 'lambda_l2': 0.03630697084016043, 'bagging_fraction': 0.8321330693588069, 'bagging_freq': 2, 'feature_fraction': 0.553876679827572, 'max_bin': 141, 'objective': 'multiclass', 'num_class': 3, 'metric': 'multi_logloss', 'n_jobs': 64, 'force_col_wise': True, 'verbose': -1}

x_cols=new_feature_cols
y_col=target
X = cv_df[x_cols].values
y = cv_df[y_col].values
dtrain = lgb.Dataset(X, label=y)

In [54]:

cv_result = lgb.cv(
    test_params, dtrain, folds=index_list, shuffle=False, metrics=None,
    feval=[
        LGBM_EvalMetrics.macro_f1_score,
        # LGBM_EvalMetrics.micro_f1_score,
        # LGBM_EvalMetrics.accuracy
        ],
    # eval_train_metric=True,
    early_stopping_rounds=20
)

In [46]:
cv_result

{'l2-mean': [0.0], 'l2-stdv': [0.0]}

In [36]:
cv_result.keys()
np.max(cv_result['micro_f1-mean'])

dict_keys(['multi_logloss-mean', 'multi_logloss-stdv', 'macro_f1-mean', 'macro_f1-stdv', 'micro_f1-mean', 'micro_f1-stdv'])

0.6026496490654853